In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import laspy
import scipy

from ofmp import superpoint_construction as sp
from ofmp import superpoint_methods as sm
from ofmp import networks as nw

In [2]:
# User Parameters
s = 0.3    # Voxel grid size
g_max = 1  # maximum ground height after flattening
c_min = 5  # shortest tree to detect
d = 0.7      # Merge radius
k = 30      # number of connections between nodes

In [3]:
tls = laspy.read('4D_7_2_03b_trans2.las')
pts = np.column_stack((tls.x, tls.y, tls.z))

# Open 3DEP Just for ground
grnd     = laspy.read('4D_7_2_3DEP_in2018.las')
grnd_pts = np.column_stack((grnd.x, grnd.y, grnd.z, grnd.classification))
grnd_pts = grnd_pts[grnd_pts[:,3]==2, 0:3] 

In [4]:
# Create the superpoint space design
design = sp.SuperpointSpaceDesign()
layer1 = design.add_design_layer(layer_name = 'layer1')
layer1.add_modifier(sm.modifier_initialize_by_cube, size = s) # voxelizes and sets centers
layer1.add_modifier(sm.modifier_center_by_com)                  # moves centers to center of mass
layer1.add_modifier(sm.modifier_center_by_nearest)              # snaps center to nearest point

# Build the space
space_constructor = sp.SuperpointSpaceConstructor(design, pts)
space = space_constructor.build_space()

In [5]:
#build space
space_constructor = sp.SuperpointSpaceConstructor(design, pts)
space = space_constructor.build_space()
network  = nw.create_network_from_superpointspace_knn(space = space, k = k, weight_method = nw.sqr_dist)

In [6]:
# ID ground superpoints and contract network
cents    = space.sp_centers
KDtree   = scipy.spatial.cKDTree(grnd_pts[:,0:2]) #build KDtree from dtm points
dist,ind = KDtree.query(cents[:,0:2], 10) #query tree with lidar points
grnd_elv = grnd_pts[ind, 2]
grnd_wts = 1/(dist+1)
terrain_at_centers = np.sum(grnd_elv*grnd_wts, axis = 1)/np.sum(grnd_wts, axis=1)
grnd_keys = np.where((cents[:,2]-terrain_at_centers) < g_max)[0].tolist() # <-----------GROUND HEIGHT
nw.contract_nodes_to_node(network, grnd_keys)

In [7]:
# Calculate shortest Path
shortest_tree = nw.pathing_single_source_all_target(network, grnd_keys[0])
forest = nw.split_into_trees(shortest_tree, grnd_keys[0])

In [8]:
# Gather points based on networks and filter by height
tree_sp_indices = {}
for key, tree in forest.items():
    indices = list(tree.nodes)
    coords = space.sp_centers[indices]
    ht_min = min(coords[:,2])
    ht_max = max(coords[:,2])
    if (ht_max - ht_min) > c_min: #<---- HEIGHT FILTER
        trunk_pos = np.mean(coords[(coords[:,2]-ht_min) < c_min], axis=0)
        tree_sp_indices[key] = {'pos':trunk_pos, 'sp':indices}

In [9]:
unvisited = dict(tree_sp_indices)  # copy so we can safely pop
clusters = []

while unvisited:
    key, data = unvisited.popitem()
    cluster_keys = [key]
    cluster_sp = list(data['sp'])
    cluster_positions = [data['pos']]

    keys_to_check = list(unvisited.keys())
    for other_key in keys_to_check:
        other_pos = unvisited[other_key]['pos']
        dist = np.linalg.norm(data['pos'] - other_pos)
        if dist < d:
            cluster_keys.append(other_key)
            cluster_sp.extend(unvisited.pop(other_key)['sp'])
            cluster_positions.append(other_pos)

    avg_pos = np.mean(cluster_positions, axis=0)
    clusters.append({'sp': cluster_sp, 'pos': avg_pos})

In [10]:
tree_number = 1
classification = np.zeros(len(pts))
for i, cluster in enumerate(clusters):
    sp_indices = cluster['sp']
    pt_indices = space.get_point_indices(sp_indices)
    classification[pt_indices] = tree_number
    tree_number += 1

In [12]:
# Write out classified tls data
las_out = '4D_7_2_03b_trans2_classed.las' 
outfile = laspy.LasData(header = tls.header)
outfile.add_extra_dim(laspy.ExtraBytesParams(
    name="tree_number",
    type=np.uint64,
    description="tree_number"
))
outfile.x     = pts[:,0]
outfile.y     = pts[:,1]
outfile.z     = pts[:,2]
outfile.tree_number = classification
outfile.write(las_out)